<a href="https://colab.research.google.com/github/AyanoNishimura/GColab_GestureRecognition/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* テストデータに偏りが無いかを確認する
* best_loss, best_epoch をループの外に置いて記録しておく
  * テストデータの学習率が一定(n = 32とか適当)epochで上がらない場合、学習を止める
  * 実装したらちゃんと動いてるか少ないデータで確認する
* 入力のsequence = 32 とする
* fc層にsequence = 1 が入るようする
* pooling層を5つ入れる
  * conv, conv, poolingとかでいい
  * 均一になるようにpooling層を入れる
* conv層を 5, 10, 15,  20 に重ねる
  * conv ksize =(3, 1), stride = (1,1) , padding=(1, 1)
  * paddingは本当は入れない方がいい
* lrは10^-2, 10^-4, 10^-6でやる
* conv層チャネル数 8, 16, 32（すべて同じでおｋ）
* weight decay を 0にする
* 36通り試す

In [0]:
# Import Chainer 
from chainer import Chain, Variable, optimizers, serializers, datasets, training, cuda
from chainer.training import extensions
import chainer.functions as F
import chainer.links as L
from chainer import Variable
import chainer

# Import NumPy and CuPy
import numpy as np
import pandas as pd
from progressbar import ProgressBar
from sklearn.model_selection import train_test_split
import math
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# mout drive
from google.colab import drive
drive.mount('/gdrive')

print('Chainer version: ', chainer.__version__)
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Chainer version:  5.0.0
GPU availability: True
cuDNN availablility: True


variables

In [0]:
is_debug = False
lr = 10 ** np.random.uniform(-6, -2)
wd = 10 ** np.random.uniform(-8, -4)
sequence = 10
batch_size = 140 # gesture_num * n
loop_time = 10
epoch_num = 200

# path
models_path = '/gdrive/My Drive/DeepLearning/GestureRecognition/models/CNN/'
output_filename = 'epoch_' + str(epoch_num) + '_lr_' + str(lr) + '_wd_' + str(wd)

models

In [0]:
class CNN(chainer.Chain):
  def __init__(self, n_out):
    super(CNN, self).__init__()
    with self.init_scope():
      self.conv1 = L.Convolution2D(None, 1000, (2, 1), stride=(1, 1), pad=(0, 0))
      self.conv2 = L.Convolution2D(None, 1500, (2, 1), stride=(1, 1), pad=(0, 0))
      self.l1 = L.Linear(None, n_out)
      
  def forward(self, x_data, t_data):
    x, t = Variable(cuda.to_gpu(x_data)), Variable(cuda.to_gpu(t_data))
#     print(x.shape, t.shape)
    
    h = self.conv1(x)
#     print(self.conv1.W.shape, self.conv1.b.shape)
#     print("a: ", h.shape)
#     h = F.local_response_normalization(h)
#     print("b: ", h.shape)
    h = F.relu(h)
#     print("c: ", h.shape)
    h = F.max_pooling_2d(h, (2, 1), stride=(2, 1))
#     print("d: ", h.shape)
    
    h = self.conv2(h)
#     print("e: ", h.shape)
#     h = F.local_response_normalization(h)
#     print("f: ", h.shape)
    h = F.relu(h)
#     print("g: ", h.shape)
    h = F.max_pooling_2d(h, (2, 1), stride=(2, 1))
#     print("h: ", h.shape)
    
    y = self.l1(h)
    
    return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

train

In [0]:
def train(file_path):
  train, test = get_train_test_data(file_path)
  print('-'*5 +'data size'+ '-'*5)
  print('train data: ', train.shape[0])
  print('test data: ', test.shape[0])
  
  t_train = pd.DataFrame({'gesture': train['gesture'], 'participants': train['participants']})
  x_train = train.drop('gesture', axis=1)
  t_test = pd.DataFrame({'gesture': test['gesture'], 'participants': test['participants']})
  x_test = test.drop('gesture', axis=1)

  train_acc_list, train_loss_list, test_acc_list, test_loss_list = training(x_train, t_train, x_test, t_test)
  
  # show graph acc
  plt.figure()
  x = np.arange(len(train_acc_list))
  plt.plot(x, train_acc_list, marker='o', label='train')
  t = np.arange(len(test_acc_list))
  plt.plot(t, test_acc_list, marker='+', label='test')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.ylim(-0.05, 1.0)
  plt.savefig(models_path + output_filename + '_acc.png')
  plt.close()
  
  plt.figure()
  x = np.arange(len(train_loss_list))
  plt.plot(x, train_loss_list, marker='o', color='r', label='train')
  t = np.arange(len(test_loss_list))
  plt.plot(t, test_loss_list, marker='+', color='b', label='test')
  plt.xlabel('epochs')
  plt.ylabel('loss')
#   plt.ylim(0.0, 10.0)
  plt.savefig(models_path + output_filename + '_loss.png')
  plt.close()

  print('Done!')

In [0]:
# 実際に学習を行う関数
def training(x_train, t_train, x_test, t_test):
  gesture_num = int(t_train['gesture'].max()) + 1
  
  print('-'*5 +'gesture num'+ '-'*5)
  print(gesture_num)
  print('-'*5 +'Hyper Parameter'+ '-'*5)
  print('lr: ', lr)
  print('weight decay: ', wd)
  
  # create model instance
  model = CNN(gesture_num)
  
  # setting GPU
  model.to_gpu()
  
  # setting optimizer
  optimizer = chainer.optimizers.Adam(alpha=lr, weight_decay_rate=wd)
  optimizer.setup(model)
  
  # training
  train_acc_list, train_loss_list = [], []
  test_acc_list, test_loss_list = [], []
  print('-'*5 +'train start'+ '-'*5)
  for epoch in range(epoch_num):
    print('Epoch: %d' % (epoch+1))
    
    # set progress bar for train
    prg = ProgressBar(0, batch_size * loop_time)
    prg_num = 0
    
    loss, accuracy = forward(x_train, t_train, gesture_num, model, optimizer, prg, prg_num)
    
    train_loss_list.append(loss)
    train_acc_list.append(accuracy)
    
    # set progress bar for test
    prg = ProgressBar(0, int(len(x_test)/sequence))
    prg_num = 0
    
    loss, accuracy = forward(x_test, t_test, gesture_num, model, optimizer, prg, prg_num, False)
    
    test_loss_list.append(loss)
    test_acc_list.append(accuracy)
    
  serializers.save_hdf5(models_path + output_filename, model)
  return train_acc_list, train_loss_list, test_acc_list, test_loss_list

In [0]:
# forwardの関数
def forward(x_data, t_data, gesture_num, model, optimizer, prg, prg_num, is_train=True):
  sum_accuracy, sum_loss, sum_batch_size = 0, 0, 0
  x_batch, t_batch = [], []
  current_gesture, batch_loop = 0, 0
  if is_train:
    batch_loop = batch_size * loop_time * sequence
  else:
    batch_loop = len(x_data)
  
  for i in range(0, batch_loop, sequence):
    if is_train: # 学習時は均等にジェスチャーが入るようにバッチを作成する
      current_gesture = int(math.floor(len(x_batch) / (batch_size / gesture_num)))
#       print('current gesture: ', current_gesture)
      index_list = t_data.loc[t_data['gesture'] == current_gesture].index
      num = np.random.randint(index_list[0], index_list[-1]-sequence)
      x = x_data.iloc[num:num+sequence]
      t = t_data.iloc[num:num+sequence]
    else: # テストデータは総なめする 
      x = x_data.iloc[i:i+sequence]
      t = t_data.iloc[i:i+sequence]

    if len(x) < sequence:
      continue

    x, t = get_sequence(x, t)

    x_batch.append(x)
    t_batch.append(t)
    
    if len(x_batch) == batch_size or i >= (batch_loop - 1) - sequence:
      x_batch = np.reshape(x_batch, (len(x_batch), 75, sequence, 1))
      t_batch = np.array(t_batch, dtype=np.int32)
      t_batch = t_batch.reshape(-1)

      # update progress bar
      prg.update(prg_num)
      prg_num = prg_num + len(x_batch)

      # get loss and accuracy
      loss, acc = model.forward(x_batch, t_batch)

      if is_train:
        model.zerograds()
        loss.backward()
        loss.unchain_backward()

        optimizer.update()

      sum_loss += float(loss.data) * len(x_batch)
      sum_accuracy += float(acc.data) * len(x_batch)
      sum_batch_size += len(x_batch)

      x_batch, t_batch = [], []
    
  # show training data loss and accuracy
  loss = sum_loss / sum_batch_size
  accuracy = sum_accuracy / sum_batch_size
  
  str = 'train'
  if not is_train:
    str = 'test'
  print(str + ' mean loss={}, accuracy={}'.format(loss, accuracy))
    
  return loss, accuracy

In [0]:
# バッチデータをフォーマットする
def get_sequence(x_batch, t_batch):
  x_batch = x_batch.drop('participants', axis=1)
  x_batch = x_batch.values.astype(np.float32)
  t_batch = t_batch.drop('participants', axis=1)
  t_batch = t_batch.mode().iloc[0]
  t_batch = t_batch.values.astype(np.int32)
  t_batch = t_batch.reshape(-1)

  return x_batch, t_batch

In [0]:
# 学習用、検証用にファイルのデータを分ける
def get_train_test_data(file_path):
  data = pd.read_csv(file_path)
  gesture_num = int(data['gesture'].max())
  train = test = pd.DataFrame().reindex_like(data)[:0]
  for index in range(0, gesture_num + 1):
    tr, te = train_test_split(data.query('gesture == ' + str(index)), test_size=0.2, shuffle=False)
    train = train.append(tr)
    test = test.append(te)
  train = train.reset_index(drop=True)
  test = test.reset_index(drop=True)
  
  # めっちゃ小さいデータを扱うとき
#   train = train[100:150]
#   train = test
  
  return train, test

main

In [0]:
if is_debug:
#   file_path = '/gdrive/My Drive/DeepLearning/GestureRecognition/data/small/train_classification.csv'
  file_path = '/gdrive/My Drive/DeepLearning/GestureRecognition/data/trim/train_classification.csv'
#   file_path = '/gdrive/My Drive/DeepLearning/GestureRecognition/data/small.csv'
else:
  file_path = '/gdrive/My Drive/DeepLearning/GestureRecognition/data/formatted/train_classification.csv'
  
train(file_path)

-----data size-----
train data:  1226407
test data:  306607
-----gesture num-----
7
-----Hyper Parameter-----
lr:  8.3675474826075e-05
weight decay:  1.3263043580379983e-05
-----train start-----
Epoch: 1


 90% (1260 of 1400) |##################  | Elapsed Time: 0:00:09 ETA:   0:00:01

train mean loss=2486.6056396484373, accuracy=0.14428572058677674


 99% (30520 of 30660) |################# | Elapsed Time: 0:01:33 ETA:   0:00:00

test mean loss=2448.333454545774, accuracy=0.13786692765953878
Epoch: 2


...

 99% (30520 of 30660) |################# | Elapsed Time: 0:01:33 ETA:   0:00:00

test mean loss=311.78127107663784, accuracy=0.21656881929609126
Epoch: 92


 90% (1260 of 1400) |##################  | Elapsed Time: 0:00:08 ETA:   0:00:00

train mean loss=296.0117218017578, accuracy=0.2600000023841858


 20% (6160 of 30660) |###                | Elapsed Time: 0:00:18 ETA:   0:01:15